In [1]:
import pandas as pd
import xlwings as xw
from xlwings.constants import LineStyle

from tkinter import *
from PIL import ImageTk, Image
from tkinter.ttk import Combobox
from tkinter import filedialog
from tkinter import ttk

In [2]:
# Allgemeine Funktionen
def open_closed_or_opened_file(path):
    """ Mit der Funktion wird zuerst versucht, die Excel-Datei mit pandas zu öffnen.
        Falls die Datei bereits von einem User geöffnet ist, wird eine Kopie erstellt.
    """
    date_format = '%d.%m.%Y'
    try:
        df = pd.read_excel(path, decimal=',')
    except:
        if '~$' in path:
            path = path.replace('~$', '')
        temp_dir = tempfile.mkdtemp()        
        temp_file_path = tempfile.mktemp(suffix='.xlsx', dir=temp_dir)
        shutil.copy2(path, temp_file_path)
        try:
            df = pd.read_excel(temp_file_path, decimal=',')  # Die temporäre Datei wird hier geöffnet
        finally:
            shutil.rmtree(temp_dir)
    return df

def decimal_value_1(value):
    """ Wandelt Zahl in Dezimalzahl mit 1 Nachkommastelle um. """
    value = float(value)
    value = "{:.2f}".format(value).replace('.', ',')
    if value[-1] == '0':
        value = value[:-1]
    return(value)

def decimal_value_2(value):
    """ Wandelt Zahl in Dezimalzahl mit 2 Nachkommastellen um. """
    value = float(value)
    value = "{:.3f}".format(value).replace('.', ',')
    if value[-1] == '0':
        value = value[:-1]
    return(value)

In [103]:
# Funktion für Go-Button
def go_dodo():
    global excel, fertig, delete_cols
    
    try:
        fertig.grid_forget()
    except:
        pass
    
    excel = excel_copy
    # Entfernen reservierter Proben
    try:
        excel = excel[excel['reserved for'].isnull()]
    except:
        pass
    try:
        excel = excel[excel['CBH reserved for'].isnull()]
    except:
        pass
    
    # Entfernen nicht relevanter Daten
    delete_cols = [col for col in delete_cols if col not in save_cols]
    excel = excel.drop(columns=delete_cols, errors='ignore')
    
    # Entfernen der Duplikate
    excel = excel.drop_duplicates()
    excel = excel.reset_index().drop('index', axis=1)
            
    # Titerangabe konvertieren
    excel['Titer M'] = ['']*len(excel)
    excel['Titer R'] = ['']*len(excel)
    titer_index = []
    for i in range(len(excel)):    
        if ':' in str(excel['Measured value'][i]):
            titer_index.append(i)
            if '<' in str(excel['Measured value'][i]):
                excel['Titer M'][i] = excel['Measured value'][i].split('<')[-1].strip()
            else:
                excel['Titer M'][i] = excel['Measured value'][i].split(':')[-1].strip()
        elif '.' in str(excel['Measured value'][i]):
            excel['Titer M'][i] = str(excel['Measured value'][i]).replace('.', ',')
        if ':' in str(excel['Reference range / Cut-off'][i]):
            if '<' in str(excel['Reference range / Cut-off'][i]):
                excel['Titer R'][i] = excel['Reference range / Cut-off'][i].split('<')[-1].strip()
            else:
                excel['Titer R'][i] = excel['Reference range / Cut-off'][i].split(':')[-1].strip()
                
    # Abgleich Messwert
    excel['Abgleich Messwert'] = ['']*len(excel)
    excel['Titer'] = ['']*len(excel)
    
    for i in titer_index:
        excel['Titer'][i] = 'x'
        try:
            if int(excel['Titer M'][i]) > int(excel['Titer R'][i]):
                excel['Abgleich Messwert'][i] = 'positiv'
                if int(excel['Titer M'][i]) < 2 * int(excel['Titer R'][i]):
                    excel['Abgleich Messwert'][i] = 'niedrig positiv'
                elif int(excel['Titer M'][i]) >= 5 * int(excel['Titer R'][i]):
                    excel['Abgleich Messwert'][i] = 'hoch positiv'
            elif int(excel['Titer M'][i]) < int(excel['Titer R'][i]):
                excel['Abgleich Messwert'][i] = 'negativ'
        except:
            pass

    # Nach Titer filtern
    # alle Parameter mit Titer in Liste wiedergeben
    excel_titer = excel.iloc[titer_index, :]
    parameter_titer = list(set(list(excel_titer['Laboratory parameter'])))
    
    # Subset für Master ID erstellen
    master_id = []
    for i in titer_index:
        if excel['Master ID'][i] not in master_id:
            master_id.append(excel['Master ID'][i])

    subset_end = pd.DataFrame(columns=list(excel.columns))
    for master in master_id:
        subset = excel[excel['Master ID'] == master]
        subset = subset[subset['Titer']=='x']
        # Subset für jeden Parameter mit Titer erstellen
        for para in parameter_titer:
            subset_para = subset[subset['Laboratory parameter']==para]
            subset_niedrig = subset_para[subset_para['Abgleich Messwert']=='niedrig positiv'].head(int(titer_input_niedrig_p.get()))
            subset_positiv = subset_para[subset_para['Abgleich Messwert']=='positiv'].head(int(titer_input_p.get()))
            subset_hoch = subset_para[subset_para['Abgleich Messwert']=='hoch positiv'].head(int(titer_input_hoch_p.get()))
            subset_negativ = subset_para[subset_para['Abgleich Messwert']=='niedrig positiv'].head(int(titer_input_n.get()))
            subset_final = pd.concat([subset_niedrig, subset_positiv, subset_hoch, subset_negativ])
            # Subsets zusammenfügen
            subset_end = pd.concat([subset_end, subset_final])
        rows_to_drop = subset[~subset.index.isin(subset_end.index)].index
        excel = excel.drop(rows_to_drop)

    # hinzugefügte Spalten wieder rauslöschen
    excel = excel.drop(['Titer M', 'Titer R', 'Abgleich Messwert', 'Titer'], axis=1)
    
    # Nach Werten filtern
    parameter = combobox.get()
    parameter_ohne_titer = parameter_liste
    if 'Alle Parameter' in parameter_ohne_titer:
        parameter_ohne_titer.remove('Alle Parameter')
    subset_parameter1 = pd.DataFrame(columns=list(excel.columns))
    subset_parameter2 = pd.DataFrame(columns=list(excel.columns))
    subset_parameter3 = pd.DataFrame(columns=list(excel.columns))
    subset_parameter4 = pd.DataFrame(columns=list(excel.columns))

    if parameter != 'Alle Parameter':
        if (input_hoch_p_wert.get() != ''):
            wert = input_hoch_p_wert.get()
            if ',' in wert:
                wert = wert.replace(',', '.')
            wert = float(wert)
            subset_parameter = excel[excel['Laboratory parameter']==parameter]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value']>wert]
            if (input_hoch_p.get() != ''):
                subset_parameter1 = subset_parameter.head(int(input_hoch_p.get()))
        if (input_p_wert1.get() != '') & (input_p_wert2.get() != ''):
            wert1 = input_p_wert1.get()
            wert2 = input_p_wert2.get()
            if ',' in wert1:
                wert1 = wert1.replace(',', '.')
            wert1 = float(wert1)
            if ',' in wert2:
                wert2 = wert2.replace(',', '.')
            wert2 = float(wert2)
            subset_parameter = excel[excel['Laboratory parameter']==parameter]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value'].between(wert1, wert2)]
            if input_p.get() != '':
                subset_parameter2 = subset_parameter.head(int(input_p.get()))
        if (input_niedrig_p_wert1.get() != '') & (input_niedrig_p_wert2.get() != ''):
            wert1 = input_niedrig_p_wert1.get()
            wert2 = input_niedrig_p_wert2.get()
            if ',' in wert1:
                wert1 = wert1.replace(',', '.')
            wert1 = float(wert1)
            if ',' in wert2:
                wert2 = wert2.replace(',', '.')
            wert2 = float(wert2)
            subset_parameter = excel[excel['Laboratory parameter']==parameter]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value'].between(wert1, wert2)]
            if input_niedrig_p.get() != '':
                subset_parameter3 = subset_parameter.head(int(input_niedrig_p.get()))
        if (input_n_wert.get() != ''):
            wert = input_n_wert.get()
            if ',' in wert:
                wert = wert.replace(',', '.')
            wert = float(wert)
            subset_parameter = excel[excel['Laboratory parameter']==parameter]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value']<wert]
            if input_n.get() != '':
                subset_parameter4 = subset_parameter.head(int(input_n.get()))
        excel = excel[excel['Laboratory parameter']!=parameter]
        excel = pd.concat([excel, subset_parameter1, subset_parameter2, subset_parameter3, subset_parameter4])
        excel = excel.reset_index(drop=True)
    else:
        if (input_hoch_p_wert.get() != ''):
            wert = input_hoch_p_wert.get()
            if ',' in wert:
                wert = wert.replace(',', '.')
            wert = float(wert)
            subset_parameter = excel[excel['Laboratory parameter'].isin(parameter_ohne_titer)]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value']>wert]
            for para in parameter_ohne_titer:
                if (input_hoch_p.get() != ''):
                    subset_parameter1 = pd.concat([subset_parameter1, subset_parameter[subset_parameter['Laboratory parameter']==para].head(int(input_hoch_p.get()))])
                else:
                    subset_parameter1 = pd.concat([subset_parameter1, subset_parameter[subset_parameter['Laboratory parameter']==para]])
        if (input_p_wert1.get() != '') & (input_p_wert2.get() != ''):
            wert1 = input_p_wert1.get()
            wert2 = input_p_wert2.get()
            if ',' in wert1:
                wert1 = wert1.replace(',', '.')
            wert1 = float(wert1)
            if ',' in wert2:
                wert2 = wert2.replace(',', '.')
            wert2 = float(wert2)
            subset_parameter = excel[excel['Laboratory parameter'].isin(parameter_ohne_titer)]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value'].between(wert1, wert2)]
            for para in parameter_ohne_titer:
                if input_p.get() != '':
                    subset_parameter2 = pd.concat([subset_parameter2, subset_parameter[subset_parameter['Laboratory parameter']==para].head(int(input_p.get()))])
                else:
                    subset_parameter2 = pd.concat([subset_parameter2, subset_parameter[subset_parameter['Laboratory parameter']==para]])
        if (input_niedrig_p_wert1.get() != '') & (input_niedrig_p_wert2.get() != ''):
            wert1 = input_niedrig_p_wert1.get()
            wert2 = input_niedrig_p_wert2.get()
            if ',' in wert1:
                wert1 = wert1.replace(',', '.')
            wert1 = float(wert1)
            if ',' in wert2:
                wert2 = wert2.replace(',', '.')
            wert2 = float(wert2)
            subset_parameter = excel[excel['Laboratory parameter'].isin(parameter_ohne_titer)]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value'].between(wert1, wert2)]
            for para in parameter_ohne_titer:
                if input_niedrig_p.get() != '':
                    subset_parameter3 = pd.concat([subset_parameter3, subset_parameter[subset_parameter['Laboratory parameter']==para].head(int(input_niedrig_p.get()))])
                else:
                    subset_parameter3 = pd.concat([subset_parameter3, subset_parameter[subset_parameter['Laboratory parameter']==para]])
        if (input_n_wert.get() != ''):
            wert = input_n_wert.get()
            if ',' in wert:
                wert = wert.replace(',', '.')
            wert = float(wert)
            subset_parameter = excel[excel['Laboratory parameter'].isin(parameter_ohne_titer)]
            # nicht numerische Werte umwandeln
            subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
            subset_parameter = subset_parameter.dropna()
            subset_parameter = subset_parameter[subset_parameter['Measured value']<wert]
            for para in parameter_ohne_titer:
                if input_n.get() != '':
                    subset_parameter4 = pd.concat([subset_parameter4, subset_parameter[subset_parameter['Laboratory parameter']==para].head(int(input_n.get()))])
                else:
                    subset_parameter4 = pd.concat([subset_parameter4, subset_parameter[subset_parameter['Laboratory parameter']==para]])
        excel = excel[~excel['Laboratory parameter'].isin(parameter_ohne_titer)]
        excel = pd.concat([excel, subset_parameter1, subset_parameter2, subset_parameter3, subset_parameter4])
        excel = excel.reset_index(drop=True)
    
    # Vorlage einlesen
    vorlage = pd.read_excel("O:\Forschung & Entwicklung\Entwicklung\Software-Entwicklung\Verfügbarkeitslisten DM\Vorlage\Vorlage_Output.xlsx", sheet_name='Output')
    vorlage_spalten = list(vorlage.iloc[8,:])
    # excel bearbeiten
    excel = excel.rename(columns={'Total volume': 'Total volume [ml]'})
    excel_spalten = list(excel.columns)
    
    # Überprüfe, ob alle Spalten in der Liste im DataFrame vorhanden sind
    added_columns = [col for col in vorlage_spalten if col not in excel_spalten]
    # Spalten, die leer sind, wurden aus excel entfernt. Diese wieder hinzufügen
    for col in added_columns:
        if col not in excel_spalten:
            excel[col] = ['']*len(excel)
    # Füge fehlende Spalten am Ende der Liste hinzu
    added_columns = [col for col in vorlage_spalten if col not in list(excel.columns)]
    new_columns_order = vorlage_spalten + added_columns
    for col in save_cols:
        new_columns_order.append(col)
    # price/ml Spalte ans Ende
    new_columns_order.remove('price/ml')
    new_columns_order.append('price/ml')
    # Wähle die Spalten des DataFrame in der neuen Reihenfolge aus
    excel = excel[new_columns_order]
    excel['price/ml'] = ['']*len(excel)
    
    # Nach Volumen filtern
    vol_min = vol_entry.get()
    vol_min = vol_min.replace(',', '.')
    if vol_min != '':
        try:
            vol_min = float(vol_min)
            excel = excel[excel['Total volume [ml]']>=vol_min]            
        except:
            pass
    excel = excel.reset_index().drop('index', axis=1)
    
    excel_copy_sum = excel.copy()
    
    # Nachkommastellen
    for i in range(len(excel)):
        if excel['Total volume [ml]'][i] != 'nan':
            try:
                value = float(excel['Total volume [ml]'][i])
                excel['Total volume [ml]'][i] = decimal_value_1(value)
            except:
                pass

        if excel['Measured value'][i] != 'nan':
            try:
                value = float(excel['Measured value'][i])
                excel['Measured value'][i] = decimal_value_2(value)
            except:
                pass
    
    # sortieren
    excel = excel.sort_values(by='Laboratory parameter')
        
    # results taken from markieren
    # Dafür Indices identifizieren
    results_index = []
    for i in range(len(excel)):
        if 'this donation' not in str(excel['results taken from'][i]):
            results_index.append(i+11)
    
    # In Excel-Vorlage übertragen
    wb = xw.Book("O:\Forschung & Entwicklung\Entwicklung\Software-Entwicklung\Verfügbarkeitslisten DM\Vorlage\Vorlage_Output.xlsx")
    sheet = wb.sheets['Output']
    sheet.range('A11').value = excel.values
    
    # zusätzliche Spalten in Vorlage einfügen
    plus_cols = [col for col in list(excel.columns) if col not in vorlage_spalten]
    plus_cols.append('price/ml')
    start_zelle = 'U10'
    referenz_zelle = sheet.range('Q10')
    # Einträge in Excel einfügen, jeweils in einer benachbarten Spalte ab der Startzelle
    for index, eintrag in enumerate(plus_cols):
        zelle = sheet.range(start_zelle).offset(column_offset=index)
        zelle.value = eintrag
        # Formatierung von Referenzzelle auf die eingefügte Zelle übertragen
        zelle.color = referenz_zelle.color
        zelle.font.size = referenz_zelle.font.size
        zelle.font.bold = referenz_zelle.font.bold
        # Rahmen von Referenzzelle auf die eingefügte Zelle übertragen
        zelle.api.Borders.LineStyle = LineStyle.xlContinuous
        zelle.api.Borders.Weight = 2
        
    # Proben ohne Referenzbereich markieren
    for i in range(len(excel)):
        # Reference range markieren
        cell = sheet.range('K{}'.format(i+11))
        cell_value = cell.value
        if cell_value is None or 'not specified' in str(cell_value):
            cell.color = (127,255,212)
        # sample condition markieren
        cell = sheet.range('D{}'.format(i+11))
        cell_value = cell.value
        if 'perfect' not in cell_value:
            cell.color = (127,255,212)
    # results taken from markieren
    if results_index != []:
        leer = Label(root, text='', bg = '#eeeee4')
        leer.grid(row=36, column=1)
        for i in results_index:
            cell = sheet.range('T{}'.format(i))
            cell.color = (127,255,212)
            
            
    # nach Parameter gruppieren
    group_para = excel_copy_sum.groupby('Laboratory parameter')['Total volume [ml]'].sum()
    group_para = pd.DataFrame(group_para)
    group_para.reset_index(inplace=True)
    group_para['Total volume [ml]'] = group_para['Total volume [ml]'].map(lambda x: x/1000)
    sheet_sum = wb.sheets['Zusammenfassung']
    sheet_sum.range('A4').value = group_para.values
    
    wb.save('Verfügbarkeitsliste.xlsx')
    
    
    fertig = Label(root, text = 'Fertig! #TeamDodo',bg = '#eeeee4', fg = '#869287',
                   font=('Ink free',10,'bold'))
    fertig.grid(row=39, column=1, columnspan=3)

In [104]:
# Funktion für Browse-Button
def select_file():
    global checkbox_vars, buttons, save_cols, delete_cols, excel_copy, vol_entry, parameter_liste
    global combobox
    global input_hoch_p
    global input_hoch_p_wert
    global input_p
    global input_p_wert1
    global input_p_wert2
    global input_niedrig_p
    global input_niedrig_p_wert1
    global input_niedrig_p_wert2
    global input_n
    global input_n_wert
    global titer_input_hoch_p
    global titer_input_niedrig_p
    global titer_input_p
    global titer_input_n
    
    try:
        fertig.grid_forget()
    except:
        pass
    
    def create_button(selected_element):
        if selected_element and selected_element not in [button['text'] for button in buttons]:
            button = Button(button_frame, text=selected_element,
                            bg='#DADEDB', font=('Ink free', 10),
                            command=lambda element=selected_element: remove_button(element))
            button.pack(anchor=W)
            buttons.append(button)
            save_cols.append(button.cget("text"))

            # Aktualisiere das Canvas-Scroll-Feld
            canvas.update_idletasks()
            canvas.config(scrollregion=canvas.bbox("all"))

    def remove_button(selected_element):
        for button in buttons:
            if button['text'] == selected_element:
                save_cols.remove(button.cget("text"))
                button.destroy()
                buttons.remove(button)
                # Entferne das Element auch aus der Dropdown-Liste
                delete_cols.append(selected_element)
                selected_element_dropdown['values'] = tuple(sorted(delete_cols))

    file_path = filedialog.askopenfilename(title="Datei auswählen")
    excel = open_closed_or_opened_file(file_path)
    excel = excel.dropna(axis=0, how='all')
    excel.columns = excel.iloc[0]
    excel = excel[1:]
    
    # Entfernen von leeren Spalten
    excel = excel.dropna(axis=1, how='all')
    
    # Kopie erstellen
    excel_copy = excel.copy()

    leer = Label(root, text='', bg='#eeeee4')
    leer.grid(row=3, column=1, columnspan=3)

    text_daten = Label(root, text='2. Entfernen nicht relevanter Daten', bg='#eeeee4',
                       font=('Ink free', 11, 'bold'))
    text_daten.grid(row=4, column=1, columnspan=3)

    delete_cols = list(excel.columns)

    # Pflichtspalten aus der Liste entfernen
    pflicht_spalten = ['Master ID', 'Pat ID', 'Total volume', 'Sample condition', 'Matrix',
                      'Gender', 'Age of proband [years]', 'Laboratory parameter', 'Measured value',
                      'Unit', 'Reference range / Cut-off', 'Remarks on reference range',
                      'Test method', 'Analytical platform', 'Manufacturer of analytical platform',
                      'Test kit', 'Manufacturer of test kit', 'price/ml', 'Freigabe',
                       'Grund der nicht erhaltenen Freigabe', 'results taken from']
    delete_cols = [spalte for spalte in delete_cols if spalte not in pflicht_spalten]
    text_mehr_entfernen = Label(root, text='Wähle hier bitte aus, falls du Spalten beibehalten möchtest', bg='#eeeee4',
                                font=('Ink free', 11))
    text_mehr_entfernen.grid(row=5, column=1, columnspan=7)

    selected_element_var = StringVar()
    selected_element_dropdown = Combobox(root, textvariable=selected_element_var, values=delete_cols,
                                          state='readonly', width=35)
    selected_element_dropdown.grid(row=6, column=1, padx=10, pady=10, columnspan=6)
    selected_element_dropdown.bind("<<ComboboxSelected>>", lambda event: create_button(selected_element_var.get()))

    buttons = []
    save_cols = []

    # Erstelle den Scrollbar-Frame
    scrollbar_frame = Frame(root, bg='#eeeee4')
    scrollbar_frame.grid(row=7, column=1, padx=5, pady=10, columnspan=5, sticky=N+S)

    # Erstelle eine vertikale Scrollbar
    scrollbar = Scrollbar(scrollbar_frame, orient=VERTICAL)
    scrollbar.pack(side=RIGHT, fill=Y)

    # Erstelle einen Canvas
    canvas = Canvas(scrollbar_frame, yscrollcommand=scrollbar.set, width=250, height=100, bg='#eeeee4')
    canvas.pack(side=LEFT, fill=BOTH, expand=True)

    # Konfiguriere die Scrollbar
    scrollbar.config(command=canvas.yview)

    # Erstelle einen Frame für die Buttons
    button_frame = Frame(canvas, bg='#eeeee4')
    canvas.create_window((0,0), window=button_frame, anchor=NW)

    # Füge dem Canvas einen Befehl zum Anpassen der Größe hinzu
    button_frame.bind("<Configure>", lambda event, canvas=canvas: canvas.configure(scrollregion=canvas.bbox("all")))

    # Option: Volumen filtern
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=17, column=1)
    text_vol_filter = Label(root, text='3. Nach Volumen filtern', bg='#eeeee4',
                            font=('Ink free', 11, 'bold'))
    text_vol_filter.grid(row=18, column=1, columnspan=2)
    text_vol = Label(root, text='Total volume [ml]', bg='#eeeee4', font=('Ink free', 11))
    text_vol.grid(row=19, column=1, columnspan=2)
    text_gr = Label(root, text='≥  ', bg='#eeeee4', font=('Ink free', 11, 'bold'))
    text_gr.grid(row=19, column=3)
    vol_entry = Entry(root, font=('Ink free', 11), width=10)
    vol_entry.grid(row=19, column=4, columnspan=3)
    
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=20, column=1)
    
    # Nach Titer filtern
    text_titer_filter = Label(root, text='4. Nach Titer filtern', bg='#eeeee4',
                             font=('Ink free', 11, 'bold'))
    text_titer_filter.grid(row=21, column=1)
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=22, column=1)
    titer_input_hoch_p = Entry(root, font=('Ink free', 11), width=5)
    titer_input_hoch_p.insert(0,'30')
    titer_input_hoch_p.grid(row=23, column=1)
    titer_text_hoch_p = Label(root, text='hochpositive Werte', bg='#eeeee4',font=('Ink free', 11))
    titer_text_hoch_p.grid(row=23, column=2, columnspan=3)
    titer_input_p = Entry(root, font=('Ink free', 11), width=5)
    titer_input_p.insert(0,'30')
    titer_input_p.grid(row=24, column=1)
    titer_text_p = Label(root, text='positive Werte', bg='#eeeee4',font=('Ink free', 11))
    titer_text_p.grid(row=24, column=2, columnspan=3)
    titer_input_niedrig_p = Entry(root, font=('Ink free', 11), width=5)
    titer_input_niedrig_p.insert(0,'30')
    titer_input_niedrig_p.grid(row=25, column=1)
    titer_text_niedrig_p = Label(root, text='niedrig positive Werte', bg='#eeeee4',font=('Ink free', 11))
    titer_text_niedrig_p.grid(row=25, column=2, columnspan=3)            
    titer_input_n = Entry(root, font=('Ink free', 11), width=5)
    titer_input_n.grid(row=26, column=1)
    titer_input_n.insert(0,'10')
    titer_text_n = Label(root, text='negative Werte', bg='#eeeee4',font=('Ink free', 11))
    titer_text_n.grid(row=26, column=2, columnspan=3)
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=27, column=1)

    # Nach niedrigsten/höchsten Werten filtern
    text_werte_filter = Label(root, text='5. Nach Werten filtern', bg='#eeeee4',
                             font=('Ink free', 11, 'bold'))
    text_werte_filter.grid(row=28, column=1)    
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=29, column=1)
    
    # mögliche Parameter
    # Parameterliste ohne Titer
    parameter_liste = []
    excel_copy = excel_copy.reset_index(drop=True)
    for i in range(len(excel_copy)):
        if ('<' not in str(excel_copy['Measured value'][i])) & ('>' not in str(excel_copy['Measured value'][i])):
            if excel_copy['Laboratory parameter'][i] not in parameter_liste:
                parameter_liste.append(excel_copy['Laboratory parameter'][i])
    parameter_liste.insert(0, 'Alle Parameter')
    
    # Dropdown
    combobox = ttk.Combobox(root, values=parameter_liste, width=50)
    # Standardwert setzen
    combobox.current(0)
    # Event-Handler für die Auswahl hinzufügen
    combobox.bind("<<ComboboxSelected>>")
    combobox.grid(row=30, column=1, columnspan=6)
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=31, column=1)
    
    input_hoch_p = Entry(root, font=('Ink free', 11), width=5)
    input_hoch_p.grid(row=32, column=1)
    text_hoch_p = Label(root, text='hochpositive Werte >', bg='#eeeee4',font=('Ink free', 11))
    text_hoch_p.grid(row=32, column=2, columnspan=2)
    input_hoch_p_wert = Entry(root, font=('Ink free', 11), width=5)
    input_hoch_p_wert.grid(row=32, column=6)
    
    input_p = Entry(root, font=('Ink free', 11), width=5)
    input_p.grid(row=33, column=1)
    text_p = Label(root, text='positive Werte', bg='#eeeee4',font=('Ink free', 11))
    text_p.grid(row=33, column=2, columnspan=2)
    text_zwischen = Label(root, text='zwischen', bg='#eeeee4',font=('Ink free', 11))
    text_zwischen.grid(row=33, column=5)
    input_p_wert1 = Entry(root, font=('Ink free', 11), width=5)
    input_p_wert1.grid(row=33, column=6)
    text_und = Label(root, text='und', bg='#eeeee4',font=('Ink free', 11))
    text_und.grid(row=33, column=7)
    input_p_wert2 = Entry(root, font=('Ink free', 11), width=5)
    input_p_wert2.grid(row=33, column=8)
    
    input_niedrig_p = Entry(root, font=('Ink free', 11), width=5)
    input_niedrig_p.grid(row=34, column=1)
    text_niedrig_p = Label(root, text='niedrig positive Werte', bg='#eeeee4',font=('Ink free', 11))
    text_niedrig_p.grid(row=34, column=2, columnspan=2)
    text_zwischen = Label(root, text='zwischen', bg='#eeeee4',font=('Ink free', 11))
    text_zwischen.grid(row=34, column=5)
    input_niedrig_p_wert1 = Entry(root, font=('Ink free', 11), width=5)
    input_niedrig_p_wert1.grid(row=34, column=6)
    text_und = Label(root, text='und', bg='#eeeee4',font=('Ink free', 11))
    text_und.grid(row=34, column=7)
    input_niedrig_p_wert2 = Entry(root, font=('Ink free', 11), width=5)
    input_niedrig_p_wert2.grid(row=34, column=8)
    
    input_n = Entry(root, font=('Ink free', 11), width=5)
    input_n.grid(row=35, column=1)
    text_n = Label(root, text='negative Werte <', bg='#eeeee4',font=('Ink free', 11))
    text_n.grid(row=35, column=2, columnspan=2)
    input_n_wert = Entry(root, font=('Ink free', 11), width=5)
    input_n_wert.grid(row=35, column=6)

    
    leer = Label(root, text='', bg = '#eeeee4')
    leer.grid(row=36, column=1)
    text_go = Label(root, text='6. Erstellen der Verfügbarkeitsliste', bg='#eeeee4',
                    font=('Ink free', 11, 'bold'))
    text_go.grid(row=37, column=1, columnspan=3)
    go = Button(root, text='   GO   ', bg='#869287', font=('Ink free', 11, 'bold'), command=go_dodo)
    go.grid(row=37, column=4)


In [105]:
root = Tk()
root.title('Verfügbarkeitslisten')
root.geometry('800x950')
root.config(bg='#eeeee4')
    
wozu = Label(root, text='Hallo DM!\n Mit diesem Programm werden die Verfügbarkeitslisten erstellt.',
             bg='#eeeee4', font=('Ink free', 15))
wozu.grid(row=1, column=1, columnspan=10)

# Auswählen der Datei
text_auswahl = Label(root, text = '1. Wähle bitte die Datei aus',
                     bg = '#eeeee4', font=('Ink free',11,'bold'))
text_auswahl.grid(row=2, column=1, columnspan=2)
browse_button = Button(root, text = 'Browse', bg = '#869287', font=('Ink free', 9, 'bold'),
                       command = select_file)
browse_button.grid(row=2, column=3)


# Dodo-Logo
frame = Frame(root, width=10, height=10)
frame.grid(row=1, column=0)
img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((100,100)), 
                 master=root,)
label = Label(frame, image=img, bg='#eeeee4')
label.pack()

root.mainloop()

C:\Users\l.nguyen quoc\AppData\Local\Temp\ipykernel_12908\3354452863.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
C:\Users\l.nguyen quoc\AppData\Local\Temp\ipykernel_12908\3354452863.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_parameter['Measured value'] = pd.to_numeric(subset_parameter['Measured value'], errors='coerce')
C:\Users\l.nguyen quoc\AppData\Local\Temp\ipykernel_12908\